In [50]:
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource,Column 
from bokeh.models import HoverTool, FuncTickFormatter, SingleIntervalTicker, LinearAxis
from bokeh.layouts import column
from IPython.display import HTML


import seaborn as sns
import random
import pandas as pd
import numpy as np

output_notebook()
%matplotlib inline

Loading BokehJS ...

In [51]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## <font color='blue'>Collision cross sections of monosaccharides </font> 

### Title: The remarkable influence of structure, partial charges and basis sets on subsequent evaluation of collision cross sections of monosaccharides 
### Authors: Lukasz G. Migas, Christopher J. Gray, Sabine L. Flitsch and Perdita E. Barran
### DOI: - not available yet

#### Here we show the collision cross sections (CCS) for each charge scheme (Mulliken, APT, MK, CHelpG, HLY and NPA) computed using MOBCAL (in nitrogen buffer gas) with partial charges computed combination of functionals (HF, B3LYP and M05) and basis sets (STO-3G, 3-21G, 6-31G, 6-31+G and 6-31G*). 

### Interactive tools:
#### <font color='green'>Hover</font>  - hover over any data point and it will bring detailed information about partial atomic charge, standard error, atom ID
#### <font color='green'>Zoom</font> - drag on the plot to zoom on specific region of interest
#### <font color='green'>Legend</font> - click on legend marker to select/deselect datasets 

In [52]:
# Pre-sets
colorSets=['#000080','#FF0080','#008000', '#FF8000', '#8009FF']

ylabelsHF=['HF/STO-3G','HF/3-21G','HF/6-31G','HF/6-31+G','HF/6-31G*']
ylabelsErrHF=['HF/STO-3G/Error','HF/3-21G/Error','HF/6-31G/Error','HF/6-31+G/Error','HF/6-31G*/Error']
ylabelsB3LYP=['B3LYP/STO-3G','B3LYP/3-21G','B3LYP/6-31G','B3LYP/6-31+G','B3LYP/6-31G*']
ylabelsErrB3LYP=['B3LYP/STO-3G/Error','B3LYP/3-21G/Error','B3LYP/6-31G/Error','B3LYP/6-31+G/Error','B3LYP/6-31G*/Error']
ylabelsM05=['M05/STO-3G','M05/3-21G','M05/6-31G','M05/6-31+G','M05/6-31G*']
ylabelsErrM05=['M05/STO-3G/Error','M05/3-21G/Error','M05/6-31G/Error','M05/6-31+G/Error','M05/6-31G*/Error']

xticksLabels = ["αGlc(1)","αGlc(2)","αGlc(3)","αGlc(4)","αGlc(5)","αGlc(6)","αGlc(7)","αGlc(8)","αGlc(9)","αGlc(10)",
                "βGlc(1)","βGlc(2)","βGlc(3)","βGlc(4)","βGlc(5)","βGlc(6)","βGlc(7)","βGlc(8)","βGlc(9)","βGlc(10)"]

# Collision cross sections - Mulliken charge scheme - Nitrogen buffer gas

In [54]:
# Import data
title = "Mulliken"
data = pd.read_csv(title+"-N2.csv", delimiter=',')
# Extract data from the table
xticks=data['Name']
xdata=data['Point']
################################ HF ################################
hoverHF = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_HF = [hoverHF,"box_zoom,pan,wheel_zoom,reset,save"]
t = figure(tools=TOOLS_HF, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="a) "+title+" - HF", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsHF)):
    legendKey=[ylabelsHF[i]]
    ydata = (data[ylabelsHF[i]])
    ydataErr=(data[ylabelsErrHF[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    t.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    t.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
t.grid.grid_line_alpha=0.7
t.title.text_font_size = '12pt'
t.legend.click_policy = "hide"
t.legend.background_fill_color = '#DCDCDC'
t.legend.location = "bottom_center"
t.legend.orientation = "horizontal"
t.grid[0].ticker.desired_num_ticks = len(xticks)
t.grid[0].ticker.num_minor_ticks = 0
t.yaxis.major_label_text_font_size = "12pt"
t.yaxis.major_label_text_font_style = 'bold'
t.yaxis.axis_label = "CCS (Å²)"
t.yaxis.axis_label_text_font_size = "14pt"
t.yaxis.axis_label_text_font_style = 'bold'
t.xaxis.major_label_text_font_size = "12pt"
t.xaxis.major_label_orientation = np.pi/4
t.xaxis.minor_tick_line_color = None
t.xaxis.major_label_text_font_style = 'bold'
t.xaxis.minor_tick_line_color = None
t.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

################################ B3LYP ################################
hoverB3LYP = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_B3LYP = [hoverB3LYP,"box_zoom,pan,wheel_zoom,reset,save"]
p = figure(tools=TOOLS_B3LYP, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="b) "+title+" - B3LYP", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsB3LYP)):
    legendKey=[ylabelsB3LYP[i]]
    ydata = (data[ylabelsB3LYP[i]])
    ydataErr=(data[ylabelsErrB3LYP[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    p.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    p.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
p.grid.grid_line_alpha=0.7
p.title.text_font_size = '12pt'
p.legend.click_policy = "hide"
p.legend.background_fill_color = '#DCDCDC'
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.grid[0].ticker.desired_num_ticks = len(xticks)
p.grid[0].ticker.num_minor_ticks = 0
p.yaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_style = 'bold'
p.yaxis.axis_label = "CCS (Å²)"
p.yaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_text_font_size = "12pt"
p.xaxis.major_label_orientation = np.pi/4
p.xaxis.minor_tick_line_color = None
p.xaxis.major_label_text_font_style = 'bold'
p.xaxis.minor_tick_line_color = None
p.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

################################ M05 ################################
hoverM05 = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_M05 = [hoverM05,"box_zoom,pan,wheel_zoom,reset,save"]
r = figure(tools=TOOLS_M05, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="c) "+title+" - M05", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsM05)):
    legendKey=[ylabelsM05[i]]
    ydata = (data[ylabelsM05[i]])
    ydataErr=(data[ylabelsErrM05[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    r.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    r.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
r.grid.grid_line_alpha=0.7
r.title.text_font_size = '12pt'
r.legend.click_policy = "hide"
r.legend.background_fill_color = '#DCDCDC'
r.legend.location = "bottom_center"
r.legend.orientation = "horizontal"
r.grid[0].ticker.desired_num_ticks = len(xticks)
r.grid[0].ticker.num_minor_ticks = 0
r.yaxis.major_label_text_font_size = "12pt"
r.yaxis.major_label_text_font_style = 'bold'
r.yaxis.axis_label = "CCS (Å²)"
r.yaxis.axis_label_text_font_size = "14pt"
r.yaxis.axis_label_text_font_style = 'bold'
r.xaxis.major_label_text_font_size = "12pt"
r.xaxis.major_label_orientation = np.pi/4
r.xaxis.minor_tick_line_color = None
r.xaxis.major_label_text_font_style = 'bold'
r.xaxis.minor_tick_line_color = None
r.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

show(column(t,p,r))

### Rotationally averaged collision cross sections calculated using the Trajectory Method with partial charges calculated using HF (a), B3LYP (b) or M05 (c) functional with several basis sets: STO-3G (<font color='#000080'>blue</font>), 3-21G (<font color='#FF0080'>magenta</font>), 6-31G (<font color='#008000'>green</font>), 6-31+G (<font color='#FF8000'>orange</font>) and 6-31G* (<font color='#8009FF'>purple</font>) for Mulliken charge scheme. Error bars on each data point represent the standard deviation from Trajectory Method calculation, typically less than 0.5 %, and always less than 1 %.

# Collision cross sections - Atomic Polar Tensors (APT) charge scheme - Nitrogen buffer gas

In [67]:
# Import data
title = "APT"
data = pd.read_csv(title+"-N2.csv", delimiter=',')
# Extract data from the table
xticks=data['Name']
xdata=data['Point']
################################ HF ################################
hoverHF = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_HF = [hoverHF,"box_zoom,pan,wheel_zoom,reset,save"]
t = figure(tools=TOOLS_HF, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="a) "+title+" - HF", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsHF)):
    legendKey=[ylabelsHF[i]]
    ydata = (data[ylabelsHF[i]])
    ydataErr=(data[ylabelsErrHF[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    t.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    t.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
t.grid.grid_line_alpha=0.7
t.title.text_font_size = '12pt'
t.legend.click_policy = "hide"
t.legend.background_fill_color = '#DCDCDC'
t.legend.location = "bottom_center"
t.legend.orientation = "horizontal"
t.grid[0].ticker.desired_num_ticks = len(xticks)
t.grid[0].ticker.num_minor_ticks = 0
t.yaxis.major_label_text_font_size = "12pt"
t.yaxis.major_label_text_font_style = 'bold'
t.yaxis.axis_label = "CCS (Å²)"
t.yaxis.axis_label_text_font_size = "14pt"
t.yaxis.axis_label_text_font_style = 'bold'
t.xaxis.major_label_text_font_size = "12pt"
t.xaxis.major_label_orientation = np.pi/4
t.xaxis.minor_tick_line_color = None
t.xaxis.major_label_text_font_style = 'bold'
t.xaxis.minor_tick_line_color = None
t.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

################################ B3LYP ################################
hoverB3LYP = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_B3LYP = [hoverB3LYP,"box_zoom,pan,wheel_zoom,reset,save"]
p = figure(tools=TOOLS_B3LYP, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="b) "+title+" - B3LYP", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsB3LYP)):
    legendKey=[ylabelsB3LYP[i]]
    ydata = (data[ylabelsB3LYP[i]])
    ydataErr=(data[ylabelsErrB3LYP[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    p.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    p.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
p.grid.grid_line_alpha=0.7
p.title.text_font_size = '12pt'
p.legend.click_policy = "hide"
p.legend.background_fill_color = '#DCDCDC'
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.grid[0].ticker.desired_num_ticks = len(xticks)
p.grid[0].ticker.num_minor_ticks = 0
p.yaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_style = 'bold'
p.yaxis.axis_label = "CCS (Å²)"
p.yaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_text_font_size = "12pt"
p.xaxis.major_label_orientation = np.pi/4
p.xaxis.minor_tick_line_color = None
p.xaxis.major_label_text_font_style = 'bold'
p.xaxis.minor_tick_line_color = None
p.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

################################ M05 ################################
hoverM05 = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_M05 = [hoverM05,"box_zoom,pan,wheel_zoom,reset,save"]
r = figure(tools=TOOLS_M05, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="c### Rotationally averaged collision cross sections calculated using the Trajectory Method with partial charges calculated using HF (a), B3LYP (b) or M05 (c) functional with several basis sets: STO-3G (<font color='#000080'>blue</font>), 3-21G (<font color='#FF0080'>magenta</font>), 6-31G (<font color='#008000'>green</font>), 6-31+G (<font color='#FF8000'>orange</font>) and 6-31G* (<font color='#8009FF'>purple</font>) for Mulliken charge scheme. Error bars on each data point represent the standard deviation from Trajectory Method calculation, typically less than 0.5 %, and always less than 1 %.) "+title+" - M05", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsM05)):
    legendKey=[ylabelsM05[i]]
    ydata = (data[ylabelsM05[i]])
    ydataErr=(data[ylabelsErrM05[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    r.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    r.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
r.grid.grid_line_alpha=0.7
r.title.text_font_size = '12pt'
r.legend.click_policy = "hide"
r.legend.background_fill_color = '#DCDCDC'
r.legend.location = "bottom_center"
r.legend.orientation = "horizontal"
r.grid[0].ticker.desired_num_ticks = len(xticks)
r.grid[0].ticker.num_minor_ticks = 0
r.yaxis.major_label_text_font_size = "12pt"
r.yaxis.major_label_text_font_style = 'bold'
r.yaxis.axis_label = "CCS (Å²)"
r.yaxis.axis_label_text_font_size = "14pt"
r.yaxis.axis_label_text_font_style = 'bold'
r.xaxis.major_label_text_font_size = "12pt"
r.xaxis.major_label_orientation = np.pi/4
r.xaxis.minor_tick_line_color = None
r.xaxis.major_label_text_font_style = 'bold'
r.xaxis.minor_tick_line_color = None
r.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

show(column(t,p,r))

### Rotationally averaged collision cross sections calculated using the Trajectory Method with partial charges calculated using HF (a), B3LYP (b) or M05 (c) functional with several basis sets: STO-3G (<font color='#000080'>blue</font>), 3-21G (<font color='#FF0080'>magenta</font>), 6-31G (<font color='#008000'>green</font>), 6-31+G (<font color='#FF8000'>orange</font>) and 6-31G* (<font color='#8009FF'>purple</font>) for APT charge scheme. Error bars on each data point represent the standard deviation from Trajectory Method calculation, typically less than 0.5 %, and always less than 1 %.

# Collision cross sections - Merz-Singh-Kollman (MK) charge scheme - Nitrogen buffer gas

In [68]:
# Import data
title = "MK"
data = pd.read_csv(title+"-N2.csv", delimiter=',')
# Extract data from the table
xticks=data['Name']
xdata=data['Point']
################################ HF ################################
hoverHF = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_HF = [hoverHF,"box_zoom,pan,wheel_zoom,reset,save"]
t = figure(tools=TOOLS_HF, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="a) "+title+" - HF", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsHF)):
    legendKey=[ylabelsHF[i]]
    ydata = (data[ylabelsHF[i]])
    ydataErr=(data[ylabelsErrHF[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    t.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    t.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
t.grid.grid_line_alpha=0.7
t.title.text_font_size = '12pt'
t.legend.click_policy = "hide"
t.legend.background_fill_color = '#DCDCDC'
t.legend.location = "bottom_center"
t.legend.orientation = "horizontal"
t.grid[0].ticker.desired_num_ticks = len(xticks)
t.grid[0].ticker.num_minor_ticks = 0
t.yaxis.major_label_text_font_size = "12pt"
t.yaxis.major_label_text_font_style = 'bold'
t.yaxis.axis_label = "CCS (Å²)"
t.yaxis.axis_label_text_font_size = "14pt"
t.yaxis.axis_label_text_font_style = 'bold'
t.xaxis.major_label_text_font_size = "12pt"
t.xaxis.major_label_orientation = np.pi/4
t.xaxis.minor_tick_line_color = None
t.xaxis.major_label_text_font_style = 'bold'
t.xaxis.minor_tick_line_color = None
t.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

################################ B3LYP ################################
hoverB3LYP = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_B3LYP = [hoverB3LYP,"box_zoom,pan,wheel_zoom,reset,save"]
p = figure(tools=TOOLS_B3LYP, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="b) "+title+" - B3LYP", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsB3LYP)):
    legendKey=[ylabelsB3LYP[i]]
    ydata = (data[ylabelsB3LYP[i]])
    ydataErr=(data[ylabelsErrB3LYP[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    p.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    p.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
p.grid.grid_line_alpha=0.7
p.title.text_font_size = '12pt'
p.legend.click_policy = "hide"
p.legend.background_fill_color = '#DCDCDC'
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.grid[0].ticker.desired_num_ticks = len(xticks)
p.grid[0].ticker.num_minor_ticks = 0
p.yaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_style = 'bold'
p.yaxis.axis_label = "CCS (Å²)"
p.yaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_text_font_size = "12pt"
p.xaxis.major_label_orientation = np.pi/4
p.xaxis.minor_tick_line_color = None
p.xaxis.major_label_text_font_style = 'bold'
p.xaxis.minor_tick_line_color = None
p.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

################################ M05 ################################
hoverM05 = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_M05 = [hoverM05,"box_zoom,pan,wheel_zoom,reset,save"]
r = figure(tools=TOOLS_M05, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="c) "+title+" - M05", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsM05)):
    legendKey=[ylabelsM05[i]]
    ydata = (data[ylabelsM05[i]])
    ydataErr=(data[ylabelsErrM05[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    r.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    r.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
r.grid.grid_line_alpha=0.7
r.title.text_font_size = '12pt'
r.legend.click_policy = "hide"
r.legend.background_fill_color = '#DCDCDC'
r.legend.location = "bottom_center"
r.legend.orientation = "horizontal"
r.grid[0].ticker.desired_num_ticks = len(xticks)
r.grid[0].ticker.num_minor_ticks = 0
r.yaxis.major_label_text_font_size = "12pt"
r.yaxis.major_label_text_font_style = 'bold'
r.yaxis.axis_label = "CCS (Å²)"
r.yaxis.axis_label_text_font_size = "14pt"
r.yaxis.axis_label_text_font_style = 'bold'
r.xaxis.major_label_text_font_size = "12pt"
r.xaxis.major_label_orientation = np.pi/4
r.xaxis.minor_tick_line_color = None
r.xaxis.major_label_text_font_style = 'bold'
r.xaxis.minor_tick_line_color = None
r.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

show(column(t,p,r))

### Rotationally averaged collision cross sections calculated using the Trajectory Method with partial charges calculated using HF (a), B3LYP (b) or M05 (c) functional with several basis sets: STO-3G (<font color='#000080'>blue</font>), 3-21G (<font color='#FF0080'>magenta</font>), 6-31G (<font color='#008000'>green</font>), 6-31+G (<font color='#FF8000'>orange</font>) and 6-31G* (<font color='#8009FF'>purple</font>) for MK charge scheme. Error bars on each data point represent the standard deviation from Trajectory Method calculation, typically less than 0.5 %, and always less than 1 %.

# Collision cross sections - CHelpG charge scheme - Nitrogen buffer gas

In [69]:
# Import data
title = "CHelpG"
data = pd.read_csv(title+"-N2.csv", delimiter=',')
# Extract data from the table
xticks=data['Name']
xdata=data['Point']
################################ HF ################################
hoverHF = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_HF = [hoverHF,"box_zoom,pan,wheel_zoom,reset,save"]
t = figure(tools=TOOLS_HF, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="a) "+title+" - HF", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsHF)):
    legendKey=[ylabelsHF[i]]
    ydata = (data[ylabelsHF[i]])
    ydataErr=(data[ylabelsErrHF[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    t.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    t.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
t.grid.grid_line_alpha=0.7
t.title.text_font_size = '12pt'
t.legend.click_policy = "hide"
t.legend.background_fill_color = '#DCDCDC'
t.legend.location = "bottom_center"
t.legend.orientation = "horizontal"
t.grid[0].ticker.desired_num_ticks = len(xticks)
t.grid[0].ticker.num_minor_ticks = 0
t.yaxis.major_label_text_font_size = "12pt"
t.yaxis.major_label_text_font_style = 'bold'
t.yaxis.axis_label = "CCS (Å²)"
t.yaxis.axis_label_text_font_size = "14pt"
t.yaxis.axis_label_text_font_style = 'bold'
t.xaxis.major_label_text_font_size = "12pt"
t.xaxis.major_label_orientation = np.pi/4
t.xaxis.minor_tick_line_color = None
t.xaxis.major_label_text_font_style = 'bold'
t.xaxis.minor_tick_line_color = None
t.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

################################ B3LYP ################################
hoverB3LYP = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_B3LYP = [hoverB3LYP,"box_zoom,pan,wheel_zoom,reset,save"]
p = figure(tools=TOOLS_B3LYP, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="b) "+title+" - B3LYP", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsB3LYP)):
    legendKey=[ylabelsB3LYP[i]]
    ydata = (data[ylabelsB3LYP[i]])
    ydataErr=(data[ylabelsErrB3LYP[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    p.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    p.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
p.grid.grid_line_alpha=0.7
p.title.text_font_size = '12pt'
p.legend.click_policy = "hide"
p.legend.background_fill_color = '#DCDCDC'
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.grid[0].ticker.desired_num_ticks = len(xticks)
p.grid[0].ticker.num_minor_ticks = 0
p.yaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_style = 'bold'
p.yaxis.axis_label = "CCS (Å²)"
p.yaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_text_font_size = "12pt"
p.xaxis.major_label_orientation = np.pi/4
p.xaxis.minor_tick_line_color = None
p.xaxis.major_label_text_font_style = 'bold'
p.xaxis.minor_tick_line_color = None
p.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

################################ M05 ################################
hoverM05 = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_M05 = [hoverM05,"box_zoom,pan,wheel_zoom,reset,save"]
r = figure(tools=TOOLS_M05, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="c) "+title+" - M05", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsM05)):
    legendKey=[ylabelsM05[i]]
    ydata = (data[ylabelsM05[i]])
    ydataErr=(data[ylabelsErrM05[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    r.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    r.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
r.grid.grid_line_alpha=0.7
r.title.text_font_size = '12pt'
r.legend.click_policy = "hide"
r.legend.background_fill_color = '#DCDCDC'
r.legend.location = "bottom_center"
r.legend.orientation = "horizontal"
r.grid[0].ticker.desired_num_ticks = len(xticks)
r.grid[0].ticker.num_minor_ticks = 0
r.yaxis.major_label_text_font_size = "12pt"
r.yaxis.major_label_text_font_style = 'bold'
r.yaxis.axis_label = "CCS (Å²)"
r.yaxis.axis_label_text_font_size = "14pt"
r.yaxis.axis_label_text_font_style = 'bold'
r.xaxis.major_label_text_font_size = "12pt"
r.xaxis.major_label_orientation = np.pi/4
r.xaxis.minor_tick_line_color = None
r.xaxis.major_label_text_font_style = 'bold'
r.xaxis.minor_tick_line_color = None
r.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

show(column(t,p,r))

### Rotationally averaged collision cross sections calculated using the Trajectory Method with partial charges calculated using HF (a), B3LYP (b) or M05 (c) functional with several basis sets: STO-3G (<font color='#000080'>blue</font>), 3-21G (<font color='#FF0080'>magenta</font>), 6-31G (<font color='#008000'>green</font>), 6-31+G (<font color='#FF8000'>orange</font>) and 6-31G* (<font color='#8009FF'>purple</font>) for CHelpG charge scheme. Error bars on each data point represent the standard deviation from Trajectory Method calculation, typically less than 0.5 %, and always less than 1 %.

# Collision cross sections - HLY charge scheme - Nitrogen buffer gas

In [70]:
# Import data
title = "HLY"
data = pd.read_csv(title+"-N2.csv", delimiter=',')
# Extract data from the table
xticks=data['Name']
xdata=data['Point']
################################ HF ################################
hoverHF = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_HF = [hoverHF,"box_zoom,pan,wheel_zoom,reset,save"]
t = figure(tools=TOOLS_HF, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="a) "+title+" - HF", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsHF)):
    legendKey=[ylabelsHF[i]]
    ydata = (data[ylabelsHF[i]])
    ydataErr=(data[ylabelsErrHF[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    t.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    t.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
t.grid.grid_line_alpha=0.7
t.title.text_font_size = '12pt'
t.legend.click_policy = "hide"
t.legend.background_fill_color = '#DCDCDC'
t.legend.location = "bottom_center"
t.legend.orientation = "horizontal"
t.grid[0].ticker.desired_num_ticks = len(xticks)
t.grid[0].ticker.num_minor_ticks = 0
t.yaxis.major_label_text_font_size = "12pt"
t.yaxis.major_label_text_font_style = 'bold'
t.yaxis.axis_label = "CCS (Å²)"
t.yaxis.axis_label_text_font_size = "14pt"
t.yaxis.axis_label_text_font_style = 'bold'
t.xaxis.major_label_text_font_size = "12pt"
t.xaxis.major_label_orientation = np.pi/4
t.xaxis.minor_tick_line_color = None
t.xaxis.major_label_text_font_style = 'bold'
t.xaxis.minor_tick_line_color = None
t.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

################################ B3LYP ################################
hoverB3LYP = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_B3LYP = [hoverB3LYP,"box_zoom,pan,wheel_zoom,reset,save"]
p = figure(tools=TOOLS_B3LYP, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="b) "+title+" - B3LYP", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsB3LYP)):
    legendKey=[ylabelsB3LYP[i]]
    ydata = (data[ylabelsB3LYP[i]])
    ydataErr=(data[ylabelsErrB3LYP[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    p.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    p.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
p.grid.grid_line_alpha=0.7
p.title.text_font_size = '12pt'
p.legend.click_policy = "hide"
p.legend.background_fill_color = '#DCDCDC'
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.grid[0].ticker.desired_num_ticks = len(xticks)
p.grid[0].ticker.num_minor_ticks = 0
p.yaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_style = 'bold'
p.yaxis.axis_label = "CCS (Å²)"
p.yaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_text_font_size = "12pt"
p.xaxis.major_label_orientation = np.pi/4
p.xaxis.minor_tick_line_color = None
p.xaxis.major_label_text_font_style = 'bold'
p.xaxis.minor_tick_line_color = None
p.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

################################ M05 ################################
hoverM05 = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_M05 = [hoverM05,"box_zoom,pan,wheel_zoom,reset,save"]
r = figure(tools=TOOLS_M05, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="c) "+title+" - M05", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsM05)):
    legendKey=[ylabelsM05[i]]
    ydata = (data[ylabelsM05[i]])
    ydataErr=(data[ylabelsErrM05[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    r.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    r.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
r.grid.grid_line_alpha=0.7
r.title.text_font_size = '12pt'
r.legend.click_policy = "hide"
r.legend.background_fill_color = '#DCDCDC'
r.legend.location = "bottom_center"
r.legend.orientation = "horizontal"
r.grid[0].ticker.desired_num_ticks = len(xticks)
r.grid[0].ticker.num_minor_ticks = 0
r.yaxis.major_label_text_font_size = "12pt"
r.yaxis.major_label_text_font_style = 'bold'
r.yaxis.axis_label = "CCS (Å²)"
r.yaxis.axis_label_text_font_size = "14pt"
r.yaxis.axis_label_text_font_style = 'bold'
r.xaxis.major_label_text_font_size = "12pt"
r.xaxis.major_label_orientation = np.pi/4
r.xaxis.minor_tick_line_color = None
r.xaxis.major_label_text_font_style = 'bold'
r.xaxis.minor_tick_line_color = None
r.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

show(column(t,p,r))

### Rotationally averaged collision cross sections calculated using the Trajectory Method with partial charges calculated using HF (a), B3LYP (b) or M05 (c) functional with several basis sets: STO-3G (<font color='#000080'>blue</font>), 3-21G (<font color='#FF0080'>magenta</font>), 6-31G (<font color='#008000'>green</font>), 6-31+G (<font color='#FF8000'>orange</font>) and 6-31G* (<font color='#8009FF'>purple</font>) for HLY charge scheme. Error bars on each data point represent the standard deviation from Trajectory Method calculation, typically less than 0.5 %, and always less than 1 %.

# Collision cross sections - Natural Population Analysis (NPA) charge scheme - Nitrogen buffer gas

In [71]:
# Import data
title = "NPA"
data = pd.read_csv(title+"-N2.csv", delimiter=',')
# Extract data from the table
xticks=data['Name']
xdata=data['Point']
################################ HF ################################
hoverHF = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_HF = [hoverHF,"box_zoom,pan,wheel_zoom,reset,save"]
t = figure(tools=TOOLS_HF, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="a) "+title+" - HF", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsHF)):
    legendKey=[ylabelsHF[i]]
    ydata = (data[ylabelsHF[i]])
    ydataErr=(data[ylabelsErrHF[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    t.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    t.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
t.grid.grid_line_alpha=0.7
t.title.text_font_size = '12pt'
t.legend.click_policy = "hide"
t.legend.background_fill_color = '#DCDCDC'
t.legend.location = "bottom_center"
t.legend.orientation = "horizontal"
t.grid[0].ticker.desired_num_ticks = len(xticks)
t.grid[0].ticker.num_minor_ticks = 0
t.yaxis.major_label_text_font_size = "12pt"
t.yaxis.major_label_text_font_style = 'bold'
t.yaxis.axis_label = "CCS (Å²)"
t.yaxis.axis_label_text_font_size = "14pt"
t.yaxis.axis_label_text_font_style = 'bold'
t.xaxis.major_label_text_font_size = "12pt"
t.xaxis.major_label_orientation = np.pi/4
t.xaxis.minor_tick_line_color = None
t.xaxis.major_label_text_font_style = 'bold'
t.xaxis.minor_tick_line_color = None
t.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

################################ B3LYP ################################
hoverB3LYP = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_B3LYP = [hoverB3LYP,"box_zoom,pan,wheel_zoom,reset,save"]
p = figure(tools=TOOLS_B3LYP, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="b) "+title+" - B3LYP", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsB3LYP)):
    legendKey=[ylabelsB3LYP[i]]
    ydata = (data[ylabelsB3LYP[i]])
    ydataErr=(data[ylabelsErrB3LYP[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    p.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    p.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
p.grid.grid_line_alpha=0.7
p.title.text_font_size = '12pt'
p.legend.click_policy = "hide"
p.legend.background_fill_color = '#DCDCDC'
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.grid[0].ticker.desired_num_ticks = len(xticks)
p.grid[0].ticker.num_minor_ticks = 0
p.yaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_style = 'bold'
p.yaxis.axis_label = "CCS (Å²)"
p.yaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_text_font_size = "12pt"
p.xaxis.major_label_orientation = np.pi/4
p.xaxis.minor_tick_line_color = None
p.xaxis.major_label_text_font_style = 'bold'
p.xaxis.minor_tick_line_color = None
p.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

################################ M05 ################################
hoverM05 = HoverTool(tooltips=[("Index", "$index"),
                              ("CCS (Å²):","@y Å²"),
                              ("Error (Å²):", "±@yErr{0.000} Å²"),
                              ("Error (%)","±@yErrPerc{0.000} %"),
                              ("Model:", "@labels"),
                              ("Method:","@chargeScheme")])
TOOLS_M05 = [hoverM05,"box_zoom,pan,wheel_zoom,reset,save"]
r = figure(tools=TOOLS_M05, plot_width=800, plot_height=300, x_range=(0.5,20.5),
           y_range=(115,150), title="c) "+title+" - M05", active_drag="box_zoom")
dataSource = []
# Prepare data for plotting
for i in range(len(ylabelsM05)):
    legendKey=[ylabelsM05[i]]
    ydata = (data[ylabelsM05[i]])
    ydataErr=(data[ylabelsErrM05[i]])
    ydataErrPerc = (ydataErr/ydata)*100
    ydataLow = ydata-ydataErr
    ydataHigh = ydata+ydataErr
    source = ColumnDataSource(data=dict(x=xdata, 
                                        y=ydata, 
                                        yLow=ydataLow,
                                        yHigh=ydataHigh,
                                        labels=xticksLabels, 
                                        yErr=ydataErr,
                                        yErrPerc=ydataErrPerc,
                                        chargeScheme=([legendKey]*(len(ydata)))))
    dataSource.append(source)
    dataColor = colorSets[i]
    r.square(x='x', y='y', size=10, fill_color=dataColor, color="black", source=source, legend=legendKey[0])
    r.segment(x0='x',x1='x', y0='yLow', y1='yHigh', color=dataColor, line_width=5, source=source, legend=legendKey[0])
labelDict = {}
for i, s in enumerate(xticks):
    labelDict[i+1] = s
r.grid.grid_line_alpha=0.7
r.title.text_font_size = '12pt'
r.legend.click_policy = "hide"
r.legend.background_fill_color = '#DCDCDC'
r.legend.location = "bottom_center"
r.legend.orientation = "horizontal"
r.grid[0].ticker.desired_num_ticks = len(xticks)
r.grid[0].ticker.num_minor_ticks = 0
r.yaxis.major_label_text_font_size = "12pt"
r.yaxis.major_label_text_font_style = 'bold'
r.yaxis.axis_label = "CCS (Å²)"
r.yaxis.axis_label_text_font_size = "14pt"
r.yaxis.axis_label_text_font_style = 'bold'
r.xaxis.major_label_text_font_size = "12pt"
r.xaxis.major_label_orientation = np.pi/4
r.xaxis.minor_tick_line_color = None
r.xaxis.major_label_text_font_style = 'bold'
r.xaxis.minor_tick_line_color = None
r.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % labelDict)

show(column(t,p,r))

### Rotationally averaged collision cross sections calculated using the Trajectory Method with partial charges calculated using HF (a), B3LYP (b) or M05 (c) functional with several basis sets: STO-3G (<font color='#000080'>blue</font>), 3-21G (<font color='#FF0080'>magenta</font>), 6-31G (<font color='#008000'>green</font>), 6-31+G (<font color='#FF8000'>orange</font>) and 6-31G* (<font color='#8009FF'>purple</font>) for NPA charge scheme. Error bars on each data point represent the standard deviation from Trajectory Method calculation, typically less than 0.5 %, and always less than 1 %.